In [1]:
import json
import itertools
import os
import re
from urllib.parse import urlsplit

import requests
from lxml import html as etree

import pdfcutter

In [2]:
BASE_URL = 'https://www.landesvertretung.bremen.de'
INDEX_URL = 'https://www.diebevollmaechtigte.bremen.de/service/bundesratsbeschluesse-17466'
PDF_URL = 'https://www.diebevollmaechtigte.bremen.de/sixcms/media.php/13/{number}.%20BR-Sitzung_Kurzbericht.pdf'

In [3]:
LINK_TEXT_RE = re.compile(r'(\d+)\. Sitzung')

def get_pdf_urls():
    response = requests.get(INDEX_URL)
    root = etree.fromstring(response.content)
    names = root.xpath('.//ul/li/a/span')
    for name in names:
        text = name.text_content()
        num = LINK_TEXT_RE.search(text)
        if num is None:
            continue
        num = int(num.group(1))
        yield num, PDF_URL.format(number=num)

In [4]:
PDF_URLS = dict(get_pdf_urls())

In [5]:
os.makedirs('./_cache', exist_ok=True)

def get_filename_url(url):
    splitresult = urlsplit(url)
    filename = splitresult.path.replace('/', '_')
    filename = os.path.join('./_cache', filename)
    if os.path.exists(filename):
        return filename
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception('{} not found'.format(url))
    with open(filename, 'wb') as f:
        f.write(response.content)
    return filename

def get_session_pdf_filename(session):
    url = PDF_URLS[session['number']]
    return get_filename_url(url)


In [6]:
with open('../bundesrat/sessions.json') as f:
    sessions = json.load(f)
len(sessions)
with open('../bundesrat/sessions_archive.json') as f:
    sessions_archive = json.load(f)
len(sessions_archive)
sessions = sessions + sessions_archive

In [7]:
def with_next(iterable):
    a, b = itertools.tee(iterable)
    next(b, None)
    return itertools.zip_longest(a, b)

In [8]:
def reformat_top_num(top_num):
    try:
        num = int(top_num)
        return top_num.zfill(3)
    except ValueError:
        return '{} {}'.format(top_num[:-1].zfill(3), top_num[-1])

def get_reformatted_tops(top_nums):
    return [reformat_top_num(t) for t in top_nums]


def get_beschluesse_text(session, filename):
    cutter = pdfcutter.PDFCutter(filename=filename)
    debugger = cutter.get_debugger()
    top_nums = [t['number'] for t in session['tops'] if t['top_type'] == 'normal']
    reformatted_top_nums = get_reformatted_tops(top_nums)

    session_number = session['number']
    page_heading = cutter.filter(search='Ergebnisse der {}. Sitzung des Bundesrates'.format(session_number))[0]

    page_number = list(cutter.filter(search='1', page=1))[-1]
    column_two = 705

    for top_num, (current, next_) in zip(top_nums, with_next(reformatted_top_nums)):
        current_top = cutter.filter(auto_regex='^{}$'.format(current))

        next_top = None
        if next_ is not None:
            next_top = cutter.filter(auto_regex='^{}$'.format(next_))

        senats = cutter.filter(auto_regex='^Senats-?') | cutter.filter(auto_regex='^Beschluss$')
        senats = senats.below(current_top)
        if next_top:
            senats = senats.above(next_top)

        ergebnis_br = cutter.filter(auto_regex='^Ergebnis BR$').below(current_top)
        if next_top:
            ergebnis_br = ergebnis_br.above(next_top)
            
        senats_text = cutter.all().filter(
            doc_top__gte=senats.doc_top,
            top__gte=page_heading.bottom,
            bottom__lt=page_number.bottom,
            right__lt=column_two
        )

        br_text = cutter.all().filter(
            doc_top__gte=ergebnis_br.doc_top,
            top__gte=page_heading.bottom,
            bottom__lt=page_number.bottom,
            right__lt=column_two
        )

        if next_top:
            br_text = br_text.above(next_top)
            senats_text = senats_text.above(ergebnis_br)
            
        senats_text = senats_text.right_of(senats)
        br_text = br_text.right_of(ergebnis_br)

        yield top_num, {'senat': senats_text.clean_text(), 'bundesrat': br_text.clean_text()}

In [9]:
def get_session(session):
    try:
        filename = get_session_pdf_filename(session)
    except KeyError:
        return
    return dict(get_beschluesse_text(session, filename))

In [10]:
FILENAME = 'session_tops.json'
if os.path.exists(FILENAME):
    with open(FILENAME) as f:
        session_tops = json.load(f)
else:
    session_tops = {}

for session in sessions:
    num = session['number']
    print('Session', num)
    if str(num) in session_tops:
        continue
    result = get_session(session)
    if result is None:
        continue
    session_tops[str(num)] = result
    with open(FILENAME, 'w') as f:
        json.dump(session_tops, f)

print('Total sessions:', len(session_tops))

Session 973


error: bad escape \s at position 1